<a href="https://colab.research.google.com/github/Jashwanth63/Custom-Model-for-classification-of-Image-Data-without-using-Convolutional-Neural-Networks/blob/main/Deep_Learning_Assignment_2022_Jashwanth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import FashionMNIST
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset

In [2]:
import pickle

def write_vectorised(data, data_usecase):
  if data_usecase == "TRAIN":
    with open("vectorised_train.txt", "wb") as file_pointer:
      pickle.dump(data, file_pointer)
  elif data_usecase == "TEST":
    with open("vectorised_test.txt","wb") as file_pointer:
      pickle.dump(data, file_pointer)
      
def read_vectorised(data_usecase):
  if data_usecase == "TRAIN":
    with open("vectorised_train.txt", "rb") as file_pointer:
      return pickle.load(file_pointer)
  elif data_usecase == "TEST":
    with open("vectorised_test.txt","rb") as file_pointer:
      return pickle.load(file_pointer)


In [3]:
def download_data(b_size=64):  
    """Download the Fashion-MNIST dataset and then load it into memory."""
    mnist_train = torchvision.datasets.FashionMNIST(".", train = True, transform = transforms.ToTensor(), target_transform= transforms.Lambda(lambda x:torch.zeros(
            10, dtype = torch.float64).scatter_(dim = 0, index= torch.tensor(x), value = 1)) ,download = True)
    mnist_train = torch.utils.data.DataLoader(mnist_train, batch_size = b_size, shuffle = True, num_workers = 3)

    mnist_test = torchvision.datasets.FashionMNIST(".", train = False, transform = transforms.ToTensor(), target_transform = transforms.Lambda(
        lambda x:torch.zeros(10, dtype = torch.float64).scatter_(dim=0, index = torch.tensor(x), value = 1)), download = True)
    mnist_test = torch.utils.data.DataLoader(mnist_test, batch_size=b_size, shuffle = True, num_workers = 3)
    return mnist_train, mnist_test  

In [4]:
training,  testing = download_data(b_size=128)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw



In [6]:
def patch_tensor_maker(image, patch_size, im_size):
  elements = []
  for l in range(0, im_size-patch_size+1, patch_size):
    for k in range(0, im_size-patch_size+1, patch_size):
      for i in range(patch_size):
        for j in range(patch_size):
          elements.append(image[l+i][k+j])
  return elements


def patch_maker(data, patch_size = 4):
  data_list_main = list()
  for i, batch in enumerate(data):
    batch_vectorised = []
    for image in batch[0]:
      im_size = image.size()[1]
      batch_vectorised.append(patch_tensor_maker(image.view(im_size, im_size), patch_size, im_size))
    main_data = torch.Tensor(batch_vectorised)
    main_labels = batch[1]
    data_list_main.append([main_data, main_labels])
    #data_list_main= TensorDataset([main_data, main_labels])
  #return torch.utils.data.DataLoader(iter(data_list_main))
  return data_list_main

In [7]:
train_data_vectorised = patch_maker(training)
test_data_vectorised = patch_maker(testing)

In [8]:
write_vectorised(train_data_vectorised, data_usecase = "TRAIN")
write_vectorised(test_data_vectorised, data_usecase = "TEST")